## Integration von Arbeitslosenzahlen auf Bundeslandebene

In [54]:
# Autor: Roman Schulze
# Datenquelle: Bundesagentur für Arbeit

In [55]:
# Bibliotheken importieren
import numpy as np
import pandas as pd

### 1. Funktionen zur Formattierung und Bereinigung der Daten

In [56]:
# Formattierung der Daten
def get_data_formatted(df):
    df.loc[0, "Unnamed: 1"] = " "
    df.drop(index=5, inplace=True)
    df.reset_index(drop=True, inplace=True)
    mask = df.loc[:, "Unnamed: 0"]  == "© Statistik der Bundesagentur für Arbeit"
    ind_pos = mask[mask==True].index[0] + 1
    df = df.iloc[0:ind_pos, :]
    df = df.rename(columns={"Unnamed: 0":"Berufsbereich","Unnamed: 1":"Berufe", 
                            "Unnamed: 2":"Anforderungsniveau", 
                            1:"Arbeitslose_Apr21"})
    df = df.iloc[:, :4]
    df["Berufsgruppe"] = df["Berufsbereich"] + " " + df["Berufe"]
    cols = ["Berufsgruppe", "Anforderungsniveau", "Arbeitslose_Apr21"]
    df = df[cols]
    df.Berufsgruppe = df.Berufsgruppe.fillna(method='ffill')
    df["Anforderungsniveau"] = df["Anforderungsniveau"].str.strip()
    return df

In [57]:
# Transformation der Daten mithilfe der Pivot-Tabelle
def transform_data(df):
    table = pd.pivot_table(df, values="Arbeitslose_Apr21", index=["Berufsgruppe"],
                    columns=["Anforderungsniveau"], aggfunc=np.sum)
    flattened = pd.DataFrame(table.to_records())
    return flattened

In [65]:
# Sortierung der Spalten & Datentypassung
def convert_data(df):
    cols = ["Bundesland", "Berufsgruppe", "Gesamt", "Helfer", "Fachkraft", "Spezialist", "Experte"]
    df = df[cols]
    # Bereinigung der Daten
    df.replace("* ", np.nan, inplace=True)
    for col in df.columns:
        if col not in ["Bundesland", "Berufsgruppe"]:
            df[col] = df[col].astype("float64")
    df = df.sort_values(by=["Bundesland", "Berufsgruppe"])
    return df

### 2. Laden und Zusammenführung der Daten

In [59]:
# Auflistung der Bundesländer
Bundesländer = ["Schleswig-Holstein", "Hamburg", "Niedersachsen", "Bremen", "Nordrhein-Westfalen", "Hessen", 
               "Rheinland-Pfalz", "Baden-Württemberg", "Bayern", "Saarland", "Berlin", "Brandenburg", 
               "Mecklenburg-Vorpommern", "Sachsen", "Sachsen-Anhalt", "Thüringen"]

In [60]:
# Gehe durch jedes Excel Blatt und ziehe die relevanten Informationen
list_of_dfs = []
for idx, val in enumerate(Bundesländer):
    df = pd.read_excel("Rohdaten_Bundesagentur_für_Arbeit_Bundesländer/Arbeitslose_Apr2021.xlsx", sheet_name="1." + str(idx + 4), header = 14, nrows=1000)
    df = get_data_formatted(df)
    df = transform_data(df)
    df["Bundesland"] = val
    list_of_dfs.append(df)

In [61]:
# Zusammenführen aller individuellen Dataframes in einen ganzen
df_all = pd.concat(list_of_dfs)

In [66]:
# finale Anpassungen
df_all = convert_data(df_all)

In [67]:
# Erste fünf Zeilen
df_all.head()

,Bundesland,Berufsgruppe,Gesamt,Helfer,Fachkraft,Spezialist,Experte
0,Baden-Württemberg,01 Angehörige der regulären Streitkräfte,17.0,NaN,12.0,NaN,5.0
1,Baden-Württemberg,"11 Land-, Tier-, Forstwirtschaftsberufe",1074.0,707.0,196.0,44.0,127.0
2,Baden-Württemberg,111 Landwirtschaft,514.0,393.0,26.0,18.0,77.0
3,Baden-Württemberg,112 Tierwirtschaft,34.0,28.0,5.0,NaN,NaN
4,Baden-Württemberg,113 Pferdewirtschaft,71.0,NaN,65.0,6.0,NaN


###### Speicherung der Daten als Excel Datei

In [69]:
# ALs Excel File speichern
df_all.to_excel("Arbeitslose_nach_BL.xlsx")  

In [ ]:
# Anmerkung: Die Ausführung des Skripts nimmt einige Zeit in Anspruch.